In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import csv
import os
from sklearn.metrics.pairwise import cosine_similarity
# Step 2: Load the datasets
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')

# Step 3: Data Preprocessing

# Prepare data by merging datasets

#MERGING DATASETS

data = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')

print(data.columns)

#Profile creation of each customer
customer_profiles = data.groupby('CustomerID').agg({
    # Average product price purchased
    'Price_y': 'mean',  
     # Total quantity purchased
    'Quantity': 'sum', 
    # Total spend
    'TotalValue': 'sum'  
}).reset_index()


# Step 4: Calculate similarity scores
# We will use the 'total_spend', 'total_quantity', and 'transaction_count' as features for similarity
X = customer_profiles[['Price_y', 'Quantity', 'TotalValue']].values

# Compute cosine similarity between customers based on the selected features
similarity_matrix = cosine_similarity(X)

# Initialize a dictionary to hold the results
results = {}

# Iterate over customers to find their top 3 lookalikes
for i, customer_id in enumerate(customer_profiles['CustomerID']):
    # Get the list of similarities with all other customers
    similarities = list(enumerate(similarity_matrix[i]))
    
    # Sort the similarities and select top 3 (excluding the customer itself)
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)[1:4]
    
    # Store the top 3 lookalikes for the customer
    results[customer_id] = [(customer_profiles.iloc[idx, 0], round(score, 2)) for idx, score in similarities]

# Ensure the output directory exists
os.makedirs('outputs', exist_ok=True)

# Write the results to a CSV file
with open('outputs/Lookalike.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    # Write the header
    writer.writerow(['CustomerID', 'LookalikeCustomerID', 'SimilarityScore'])
    
    # Write the results for each customer
    for customer_id, lookalikes in results.items():
        for lookalike in lookalikes:
            # Write each lookalike and similarity score as separate rows
            writer.writerow([customer_id, lookalike[0], lookalike[1]])

# Display the first few rows of the lookalike recommendations
lookalike_df.head()

Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'CustomerName', 'Region',
       'SignupDate', 'ProductName', 'Category', 'Price_y'],
      dtype='object')


,CustomerID,LookalikeCustomerID,SimilarityScore
0,C0001,C0101,1.0
1,C0001,C0131,1.0
2,C0001,C0063,1.0
3,C0002,C0029,1.0
4,C0002,C0034,1.0
